In [1]:
import sys 
import cv2
import os
from os.path import isfile, join
import pandas as pd
from youtubesearchpython import SearchVideos
import pandas as pd
from lxml import html, etree
import pafy
import json
import os
import numpy as np
from ffpyplayer.player import MediaPlayer
from pytube import YouTube
from moviepy.editor import *
from moviepy.video.VideoClip import VideoClip

# STEP1 - Download YouTube by url

In [ ]:
url = 'https://www.youtube.com/watch?v=GGOMD2DlJUY&t=1s'
video = pafy.new(url)
bestResolutionVideo = video.getbest()
bestResolutionVideo.download()

# STEP2 - (May refresh kernel) - make sure the fps is NOT zero 

In [ ]:
# After download, restart the kernel, and make sure this is not 0.0
vidcap = cv2.VideoCapture('Best Clown Pranks Compilation 2018.mp4')
vidcap.get(cv2.CAP_PROP_FPS)
#vidcap.get(cv2.CAP_PROP_POS_MSEC)

# STEP3 - Parse the video 

In [ ]:
# video - to -images 
vidcap = cv2.VideoCapture('Best Clown Pranks Compilation 2018.mp4')
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)   #set the capturing start at (sec*1000 milliseconds)
    hasFrames,image = vidcap.read()
    if hasFrames:
        os.chdir(r'C:\Users\DrHeh\Google Drive\programming\py_InsightFellowProject\pre-course prep\YouTubeVideos')
        cv2.imwrite("image"+str(count)+".jpg", image)     # save frame as JPG file
    return hasFrames


if __name__ == '__main__':
    sec = 104 #capture start at 120 seconds 
    frameRate = 1/vidcap.get(cv2.CAP_PROP_FPS) #//it will capture image in each "frameRate" second
    count=1
    success = getFrame(sec)
    while success:
        while (count < 500):
            count = count + 1  #for label images 
            #sec = sec + 1 #every 1 sec 
            sec = sec + frameRate*1 #every 1 frames
            sec = round(sec, 2)
            success = getFrame(sec)
        else:
            break

# STEP 4 Parse Audio

In [ ]:
audio_cut = VideoFileClip(bestResolutionVideo.filename).subclip(104,120) #second start, end
audio_cut.write_videofile('audio.mp4', audio=True)

# After MASK RCNN, PostStep - Construct Video

In [ ]:
# images - to -videos 
pathIn= '/home/ubuntu/Mask_RCNN/data/PreVideoFrame/'
pathOut = '/home/ubuntu/Mask_RCNN/data/videoConstruct1.mp4'
df_filename_imageID = pd.read_csv('/home/ubuntu/Mask_RCNN/data/TestSampleImageID.txt')
fps = 1/30 #(or vidcap.get(cv2.CAP_PROP_FPS))
   
# start to convert images to videos
frame_array = []
for file in df_filename_imageID['Index_outputFile']:
    filename = pathIn + str(file) +'.jpg'
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()

# Final Step - Play the Video

In [2]:
# play video 
video_path = 'videoConstruct1.mp4' 
audio_path = "part.mp4"
def PlayVideo(video_path):
    video = cv2.VideoCapture(video_path)
    player = MediaPlayer(audio_path)
    while True:
        grabbed, frame=video.read()
        audio_frame, val = player.get_frame()
        if not grabbed:
            print("End of video")
            break
        if cv2.waitKey(28) & 0xFF == ord("q"):
            break
        #frame = cv2.resize(frame, (1800, 1000))
        cv2.imshow("Video", frame)
        if val != 'eof' and audio_frame is not None:
            #audio
            img, t = audio_frame
    video.release()
    cv2.destroyAllWindows()
PlayVideo(video_path)

End of video
